In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
sns.set()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2. ESPECIALIZACIÓN EN IA/CLASES UAO COLAB/Herramientas/PROYECTOS/Proyecto 3 regresión logistica/datasets/winequality-red.csv', 
                 sep=';')


In [ ]:
estado = []
for i in df['quality']: 
    if i >= 6:
        i = 1
        estado.append(i)
    else:
        i = 0
        estado.append(i)
df['estado'] = estado 
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,estado
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,0
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,0
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,0


In [ ]:
df.isna().sum()


fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
estado                  0
dtype: int64

In [ ]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free sulfur dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total sulfur dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


ver si meter outliersAQUI 

In [ ]:
seaborn.pairplot(df, hue = "estado")
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# the target column (in this case 'grade') should not be included in variables
#Categorical variables may or maynot be added if any


variables = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',]]
x = add_constant(variables)
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(x.values,i) for i in range (x.shape[1])]
vif['features'] = x.columns
vif

,VIF,features
0,1.710538e+06,const
1,7.767512e+00,fixed acidity
2,1.789390e+00,volatile acidity
3,3.128022e+00,citric acid
4,1.702588e+00,residual sugar
5,1.481932e+00,chlorides
6,1.963019e+00,free sulfur dioxide
7,2.186813e+00,total sulfur dioxide
8,6.343760e+00,density
9,3.329732e+00,pH


In [ ]:
x = df.drop('estado', axis =1 )
y = df['estado']

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x)
scaled_x = scaler.transform(x)

REGRESIÓN LOGISTICA

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_x, y , test_size = 0.3)

In [ ]:
reg = LogisticRegression() 
reg.fit(x_train,y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_hat = reg.predict(x_test) # y_hat holding the prediction made with the algorithm using x_test

In [ ]:
acc = metrics.accuracy_score(y_hat,y_test)# To know the accuracy
acc

1.0

In [ ]:
reg.intercept_ # Intercept of the regression

array([-7.72893805])

In [ ]:
reg.coef_ # coefficients of the variables / features 

array([[ 0.1617105 , -1.53598592, -0.13196723,  0.17834669, -0.30937825,
         0.22890698, -1.28165436, -0.37183744,  0.34562887,  1.2646259 ,
         2.17758426, 15.03089346]])

In [ ]:
result = pd.DataFrame(data = x.columns, columns = ['Features'])
result['weight'] = np.transpose(reg.coef_)
result['odds'] = np.exp(np.transpose(reg.coef_))
result

,Features,weight,odds
0,fixed acidity,0.161711,1.175520e+00
1,volatile acidity,-1.535986,2.152434e-01
2,citric acid,-0.131967,8.763697e-01
3,residual sugar,0.178347,1.195240e+00
4,chlorides,-0.309378,7.339031e-01
5,free sulfur dioxide,0.228907,1.257225e+00
6,total sulfur dioxide,-1.281654,2.775777e-01
7,density,-0.371837,6.894663e-01
8,pH,0.345629,1.412878e+00
9,sulphates,1.264626,3.541768e+00


In [ ]:
cm = confusion_matrix(y_hat,y_test)
cm

array([[246,   0],
       [  0, 234]])

In [ ]:
# Format for easier understanding
cm_df = pd.DataFrame(cm)
cm_df.columns = ['Predicted 0','Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0',1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,246,0
Actual 1,0,234


In [ ]:
all_predict = reg.predict(x_test)
print(all_predict)

print("Datos de Validacion\n", y_test)


[1 0 0 1 1 0 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0
 0 1 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0
 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0
 0 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1
 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 0
 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 0
 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0
 0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 1 0
 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 1 1 1 1 0 0 0 1 1
 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 1 1 0 1 0 0
 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 0 0 1 1 1 1]
Datos de Validacion
 366  

hacer pruebas con más datos


cambiar el estado > 6.5